In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Load dataset
df = pd.read_csv('superstore.csv')

df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(columns=['记录数'], inplace=True)

In [ ]:
df.head()

In [ ]:
import pandas as pd

# Load dataset

# Rename for consistency
df.rename(columns={'Order.Date': 'order_date', 'Market': 'market', 'City': 'city', 'Country': 'country'}, inplace=True)

# Display dataset info
print(df.info())


In [ ]:
# Convert 'order_date' to datetime format
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')

# Check if the conversion was successful
print(df['order_date'].dtype)  # Should print 'datetime64[ns]'

# Now you can apply datetime functions
df['is_weekend'] = df['order_date'].dt.weekday.isin([5, 6]).astype(int)  # 5 = Saturday, 6 = Sunday


df.rename(columns={'Order.Date': 'order_date'}, inplace=True)

# 🔹 Add `is_weekend`
df['is_weekend'] = df['order_date'].dt.weekday.isin([5, 6]).astype(int)  # 5 = Saturday, 6 = Sunday

# 🔹 Add `month`, `quarter`
df['month'] = df['order_date'].dt.month
df['quarter'] = df['order_date'].dt.quarter

# 🔹 Add `week_of_year`
df['week_of_year'] = df['order_date'].dt.isocalendar().week

# 🔹 Add `season`
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df['season'] = df['month'].apply(get_season)

# 🔹 Add `is_holiday`
# Define a list of known holidays (can be expanded)
holidays = {
    'New Year': '01-01',
    'Christmas': '12-25',
    'Independence Day': '07-04',
    'Thanksgiving': '11-23',
    'Labor Day': '09-01',
}

# Function to check if a date is a holiday
def is_holiday(date):
    date_str = date.strftime("%m-%d")
    return int(date_str in holidays.values())

df['is_holiday'] = df['order_date'].apply(is_holiday)

# 🔹 Add `days_before_holiday` and `days_after_holiday`
holiday_dates = [datetime(df['order_date'].dt.year.min(), int(h.split('-')[0]), int(h.split('-')[1])) for h in holidays.values()]

def get_days_to_holiday(date, direction=1):
    """ direction = 1 for future holidays, -1 for past holidays """
    if direction == 1:
        future_holidays = [h for h in holiday_dates if h >= date]
        return (min(future_holidays, default=date) - date).days if future_holidays else np.nan
    else:
        past_holidays = [h for h in holiday_dates if h <= date]
        return (date - max(past_holidays, default=date)).days if past_holidays else np.nan

df['days_before_holiday'] = df['order_date'].apply(lambda x: get_days_to_holiday(x, 1))
df['days_after_holiday'] = df['order_date'].apply(lambda x: get_days_to_holiday(x, -1))

# 🔹 Display Updated Data
print(df[['order_date', 'is_weekend', 'month', 'quarter', 'season', 'is_holiday', 'days_before_holiday', 'days_after_holiday', 'week_of_year']].head())

In [ ]:
import pandas as pd

pd.set_option('display.max_columns', None)
df.head()

In [ ]:
import requests
import pandas as pd

def fetch_oil_prices_alpha_vantage(api_key):
    url = f"https://www.alphavantage.co/query?function=WTI&apikey={api_key}"

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if "data" in data:
            df = pd.DataFrame(data["data"])
            df = df.rename(columns={"date": "order_date", "value": "oil_price"})
            df["order_date"] = pd.to_datetime(df["order_date"])
            return df
        else:
            print("No valid data found in response.")
    else:
        print(f"Error fetching data: {response.status_code}")
    return pd.DataFrame()

# Replace with your Alpha Vantage API Key
alpha_vantage_api_key = "9QTSDG5457V95M10"
oil_prices_df = fetch_oil_prices_alpha_vantage(alpha_vantage_api_key)

print(oil_prices_df.head())


In [ ]:
oil_prices_df = pd.read_csv('crude-oil-price.csv')

In [ ]:
oil_prices_df = oil_prices_df.rename(columns={"date": "order_date", "price": "oil_price"})


In [ ]:
df["order_date"] = pd.to_datetime(df["order_date"], errors="coerce")
oil_prices_df["order_date"] = pd.to_datetime(oil_prices_df["order_date"], errors="coerce")


In [ ]:
oil_prices_df["order_date"] = oil_prices_df["order_date"].dt.tz_localize(None)


In [ ]:
df = df.drop(columns=[col for col in df.columns if "oil_price" in col], errors="ignore")


In [ ]:
df = df.merge(oil_prices_df[["order_date", "oil_price"]], on="order_date", how="left")


In [ ]:
df = df.merge(oil_prices_df[["order_date", "oil_price"]], on="order_date", how="left")


In [ ]:
print(df.columns)


In [ ]:
df = df.drop(columns=[col for col in df.columns if "oil_price" in col], errors="ignore")


In [ ]:
df = df.merge(oil_prices_df[["order_date", "oil_price"]], on="order_date", how="left")


In [ ]:
df.head()


In [ ]:
oil_df = pd.read_csv('crude-oil-price.csv')

In [ ]:
oil_df.head()

In [ ]:
import pandas as pd

# Convert date columns to datetime format
df['order_date'] = pd.to_datetime(df['order_date'])
oil_df['date'] = pd.to_datetime(oil_df['date'])

# Extract year and month for merging
df['year'] = df['order_date'].dt.year
df['month'] = df['order_date'].dt.month

oil_df['year'] = oil_df['date'].dt.year
oil_df['month'] = oil_df['date'].dt.month

# Merge the dataframes on year and month
df = df.merge(oil_df[['year', 'month', 'price']], on=['year', 'month'], how='left')

# Rename 'price' to 'oil_price' for clarity
df.rename(columns={'price': 'oil_price'}, inplace=True)

# Display result
df.head()


In [ ]:
df.drop(columns=['oil_price_x',	'oil_price_y',	'year'], inplace=True)

In [ ]:
df.head()

In [ ]:
inflation_df = pd.read_csv('US_inflation_rates.csv')

In [ ]:
# Convert inflation_df date column to datetime format
inflation_df['date'] = pd.to_datetime(inflation_df['date'])

# Extract year and month for merging
inflation_df['year'] = inflation_df['date'].dt.year
inflation_df['month'] = inflation_df['date'].dt.month

# Merge inflation rate into df
df = df.merge(inflation_df[['year', 'month', 'value']], on=['year', 'month'], how='left')

# Rename 'value' to 'inflation_rate' for clarity
df.rename(columns={'value': 'inflation_rate'}, inplace=True)

# Display result
df.head()


In [ ]:
print(inflation_df.columns)


In [ ]:
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')  # Ensure datetime format
df['year'] = df['order_date'].dt.year
df['month'] = df['order_date'].dt.month


In [ ]:
print(inflation_df.head())


In [ ]:
df = df.merge(inflation_df[['year', 'month', 'value']], on=['year', 'month'], how='left')
df.rename(columns={'value': 'inflation_rate'}, inplace=True)


In [ ]:
df.head()

In [ ]:
df.drop(columns=['weather'], inplace=True)

In [ ]:
df.to_csv("cached_oil_prices.csv", index=False)